In [1]:
import requests,json,config,pickle,time



class VCrequestClass:
    api_key=""
    payload={}
    headers ={}
    
    def __init__(self):
        self.api_key=config.api_key
    def _glupi_hash(self,x,y,radius,type):
        return str(x)+'|'+str(y)+str(radius)+type
    def request(self,x,y,radius,type):
       
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={x}%2C{y}&radius={radius}&type={type}&key={self.api_key}"
        with open('cache.pkl', 'rb') as f:
            cache = pickle.load(f)
        if(self._glupi_hash(x,y,radius,type) in cache):
            #print('uzeto iz cache')
            return cache[self._glupi_hash(x,y,radius,type)]
        else:
            l=[]
            time.sleep(2)
            
            page1 = requests.request("GET", url, headers=self.headers, data=self.payload)
            page1_dict=json.loads(page1.text)
            
            l+=page1_dict['results']
            broj=1
            if('next_page_token' in page1_dict):
              
                broj=2
                next_page=page1_dict['next_page_token']
                time.sleep(2)
                url2 = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page}&key={config.api_key}"
                page2 = requests.request("GET",url2, headers=self.headers, data=self.payload)
                page2_dict=json.loads(page2.text)
                l+=page2_dict['results']
                if('next_page_token' in page2_dict):
                    broj=3
                    time.sleep(2)
                    next_page=page2_dict['next_page_token']
                    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page}&key={config.api_key}"
                    page3 = requests.request("GET", url, headers=self.headers, data=self.payload)
                    page3_dict=json.loads(page3.text)
                    l+=page3_dict['results']

            cache[self._glupi_hash(x,y,radius,type)]=l
            
            with open('cache.pkl', 'wb') as f:
                pickle.dump(cache, f)
            
       


In [7]:
import random
from ipyleaflet import *
from ipywidgets import HTML
import geographiclib.geodesic as geodesic
import math 
import csv

x=44.01115533984336 #KRAGUJEVAC
y=20.910337884603837
radius=10000
m=Map(center=(x,y),zoom=12,scroll_wheel_zoom=True)

a = 6378137
f = 1 / 298.257223563
g = geodesic.Geodesic(a,f)

csv_file_path = 'data.csv'

m=Map(center=(x,y),zoom=12,scroll_wheel_zoom=True)

class VCKrug:
    x=0
    y=0
    radius=0
    def __init__(self,x,y,radius):
       self.x=x
       self.y=y
       self.radius=radius
    def sketch(self,color):
        circle = Circle()
        circle.location = (self.x,self.y)
        circle.radius=int(self.radius)
        circle.fill_opacity=0
        circle.color=color
        global m
        m.add_layer(circle)
    def print(self):
        print(self.x,self.y,self.radius)


def new_circles(krug):
    res=[VCKrug(krug.x,krug.y,krug.radius/2)]
    for i in range(6):
        new_cor=g.Direct(x, y, i*60, math.sqrt(3)*(krug.radius/2))
        res.append(VCKrug(new_cor['lat2'],new_cor['lon2'],krug.radius/2))
    return res


q=[]




pocetni_krug=VCKrug(x,y,radius)
pocetni_krug.sketch("red")
q.append(pocetni_krug)
count=0

def check():
    global count
    count+=1
    if(count>19) :
        return 0
    else:
        return 1
    

boje=['red','green','blue','yellow','black']
broj=0
brojkafica=0
v = VCrequestClass() 
while(len(q)>0):
    element=q[0]
    broj+=1
    
    element.sketch(color='black')
    q=q[1:]
   
    z=v.request(element.x,element.y,element.radius,'restaurant')
 
    
    if(z is not None):
        for i in z:
            center = (i['geometry']['location']['lat'],i['geometry']['location']['lng'])
            print(i)

            marker = Marker(location=center, draggable=False)
            message2 = HTML()
            message2.value = i['name']
            popup = Popup(
            location=center,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
            )
            marker.popup = message2

            m.add_layer(marker)


        brojkafica+=len(z)
        #print(len(z))
        if(len(z)==60):
            #print('yes')
            for i in new_circles(element):
                q.append(i)
        else:
            pass
            #print("no")
print(brojkafica)
m


{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 44.0259971, 'lng': 20.9201717}, 'viewport': {'northeast': {'lat': 44.0273342302915, 'lng': 20.9217336302915}, 'southwest': {'lat': 44.0246362697085, 'lng': 20.9190356697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Ženeva Lux', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110774390126121146417">relja kocic</a>'], 'photo_reference': 'AZose0n35Rm2tIgnezwDxh4W1no_XOk21zRvA3JaFXlR8YzbHXABg0RqO_XVgMH-t-I0kxaniTkq8U6W2k4QyutvfXNneTFQmSKl-aoWaGQjlculldg6DUZY2kqE8mkO4pR-eWNwQUjxd-BvgFwHMToIBUHej67YmoVCYaKiiFrYAJQ7J7_5', 'width': 6000}], 'place_id': 'ChIJ5yZDK9MmV0cRp9JD0jbWLaY', 'plus_code': {'compound_code': '2WGC+93 Kragujevac, Serbia', 'global_code':

Map(center=[44.01115533984336, 20.910337884603837], controls=(ZoomControl(options=['position', 'zoom_in_text',…